In [61]:
using System.IO;

In [63]:
public string MachReadFromN(byte[] page,int start_offset,int length)
{
	var ret = page[start_offset..(start_offset+length)];
	return Convert.ToHexString(ret).ToLower();
}
public void GetInnodbPageType()
{
	var f=new FileInfo(@"E:\MySQL\Data\admin\tb_users.ibd");



	var fsize = f.Length/INNODB_PAGE_SIZE;
	dynamic ret = new{};
	var read = new BinaryReader(f.OpenRead());
	for (int i = 0;i<fsize;i++)
	{
		
		var page = read.ReadBytes(INNODB_PAGE_SIZE);
		//file header
		var page_offset = MachReadFromN(page,FIL_PAGE_OFFSET,4);
		var page_pre = MachReadFromN(page,FIL_PAGE_OFFSET+4,4);
		var page_next = MachReadFromN(page,FIL_PAGE_OFFSET+8,4);
		var page_lsn = MachReadFromN(page,FIL_PAGE_OFFSET+12,8);
		
		var page_type = MachReadFromN(page,FIL_PAGE_TYPE,2);
		var flush_lsn = MachReadFromN(page,FIL_PAGE_TYPE+2,8);
		var space_id = MachReadFromN(page,FIL_PAGE_TYPE+10,4);
		

		display($"page offset [{page_offset}], page type <{innodb_page_type[page_type]}>, pre page [{page_pre}], next page [{page_next}] , flush_lsn <{flush_lsn}> , space_id <{space_id}>");


		//page header
		var page_dir_slots = MachReadFromN(page,FIL_PAGE_DATA,2);
		var page_heap_top = MachReadFromN(page,FIL_PAGE_DATA+2,2);
		var page_n_heap = MachReadFromN(page,FIL_PAGE_DATA+4,2);
		var	page_level = MachReadFromN(page,FIL_PAGE_DATA+26,2);
		var	page_index_id = MachReadFromN(page,FIL_PAGE_DATA+28,8);
		var page_btr_seg_leaf = MachReadFromN(page,FIL_PAGE_DATA+36,10);
		var page_btr_seg_top = MachReadFromN(page,FIL_PAGE_DATA+46,10);

		display($"page_dir_slots {page_dir_slots}  page_heap_top {page_heap_top}  page_n_heap {page_n_heap} page_level {page_level} page_index_id {page_index_id} page_btr_seg_leaf {page_btr_seg_leaf} page_btr_seg_top {page_btr_seg_top}");
		//Infimum + Supremum


		//User Records

		// Free Space

		// Page Directory
		var data = MachReadFromN(page,38,INNODB_PAGE_SIZE-8-32);

		//File Trailer
		var file_trailer = MachReadFromN(page,INNODB_PAGE_SIZE-8,8);

			
		
	}
}
GetInnodbPageType();

page offset [00000000], page type <File Space Header>, pre page [00000000], next page [00000000] , flush_lsn <0000000000000000> , space_id <000005e1>

page_dir_slots 0000  page_heap_top 05e1  page_n_heap 0000 page_level 0000 page_index_id ffffffff0000ffff page_btr_seg_leaf ffff0000000000010000 page_btr_seg_top 0000009e00000000009e

page offset [00000001], page type <Insert Buffer Bitmap>, pre page [00000000], next page [00000000] , flush_lsn <0000000000000000> , space_id <000005e1>

page_dir_slots 0000  page_heap_top 0000  page_n_heap 0000 page_level 0000 page_index_id 0000000000000000 page_btr_seg_leaf 00000000000000000000 page_btr_seg_top 00000000000000000000

page offset [00000002], page type <File Segment inode>, pre page [00000000], next page [00000000] , flush_lsn <0000000000000000> , space_id <000005e1>

...

page_dir_slots 0002  page_heap_top 0078  page_n_heap 8002 page_level 0000 page_index_id 0000000000000725 page_btr_seg_leaf 000005e1000000020572 page_btr_seg_top 000005e10000000204b2

page offset [00000000], page type <Freshly Allocated Page>, pre page [00000000], next page [00000000] , flush_lsn <0000000000000000> , space_id <00000000>

page_dir_slots 0000  page_heap_top 0000  page_n_heap 0000 page_level 0000 page_index_id 0000000000000000 page_btr_seg_leaf 00000000000000000000 page_btr_seg_top 00000000000000000000

page offset [00000000], page type <Freshly Allocated Page>, pre page [00000000], next page [00000000] , flush_lsn <0000000000000000> , space_id <00000000>

page_dir_slots 0000  page_heap_top 0000  page_n_heap 0000 page_level 0000 page_index_id 0000000000000000 page_btr_seg_leaf 00000000000000000000 page_btr_seg_top 00000000000000000000